# D-Score Steamflow Analysis Workflow

<img src='./Eval_Analysis.svg' width=600>

:::{note}

_This notebook adapted from originals by Timothy Hodson and Rich Signell. See that upstream work at:_
* https://github.com/thodson-usgs/dscore
* https://github.com/USGS-python/hytest-evaluation-workflows/

:::


## Essential Benchmark Components
This benchmark notebook will present a workflow which follows a canonical model for Essential Benchmark Components: 
1) A set of predictions and matching observations; 
2) The domain (e.g. space or time) over which to benchmark;
3) A set of statistical metrics with which to benchmark. 

Let's get started.

## Step 0: Load libraries and configure Python computing environment

In [1]:
import pandas as pd
import logging

## Step 1: Load Data

<img src='./Eval_Analysis_Data.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  <span style="color:red; font-size:large"><<--You are here</span>
2) The domain over which to benchmark 
3) A set of statistical metrics with which to benchmark. 

Finding and loading data is made easier for this particular workflow (the _streamflow_ variable), in that most of it has been 
pre-processed and stored in a cloud-friendly format.  That process is described in the [data preparation](./01_Data_Prep.ipynb)
notebook. We will proceed here using the already-prepared data for _streamflow_, which is included in the HyTEST **intake catalog**.  

:::{sidebar}
Learn more about `intake` [here](/dev/null)
:::

In [2]:
import intake
cat = intake.open_catalog(r'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml')
print("Available datasets: \n", "\n".join(cat.keys()))

Available datasets: 
 conus404-hourly-onprem
conus404-hourly-cloud
conus404-daily-onprem
conus404-daily-diagnostic-onprem
conus404-daily-cloud
conus404-daily-diagnostic-cloud
conus404-monthly-onprem
conus404-monthly-cloud
nwis-streamflow-usgs-gages-onprem
nwis-streamflow-usgs-gages-cloud
nwm21-streamflow-usgs-gages-onprem
nwm21-streamflow-usgs-gages-cloud
nwm21-streamflow-cloud
nwm21-scores
lcmap-cloud
conus404-hourly-cloud-dev
nhm-v1.0-daymet-byHRU-onprem
nhm-v1.0-daymet-byHW-musk-onprem
nhm-v1.0-daymet-byHW-musk-obs-onprem
nhm-v1.0-daymet-byHW-noroute-onprem
nhm-v1.0-daymet-byHW-noroute_obs-onprem
nhm-v1.1-gridmet-byHRU-onprem
nhm-v1.1-gridmet-byHW-onprem
nhm-v1.1-gridmet-byHWobs-onprem



The above list represents the processed datasets available for benchmarking.  If a dataset
you want is not in that list, 
you can load data directly from S3 or onprem. 
If you load data from a source other than this list, you can jump to [Step 2: Restrict to a Domain](#step-2-restrict-to-a-domain)

Note that the interesting datasets in the cataloged dataset above are duplicated: Some are `-onprem` 
and some are `-cloud`. Same data, but the storage location and access protocol will be different. You 
will definitely want to use the correct copy of the data for your computing environment.  
* `onprem` : Direct access via the `caldera` filesystem from _denali_ or _tallgrass_
* `cloud` : Network access via S3 bucket, suitable for consumption on cloud-hosted jupyter servers. You could also access using any network-attached computer, but the amount of data will likely saturate your connection.  Use in the cloud (e.g. ESIP QHub)

So... are you on-prem? 

In [3]:
import platform
onprem = (platform.node() in ['denali', 'tallgrass'])
if onprem:
    print("Yes : -onprem")
    obs_data_src='nwis-streamflow-usgs-gages-onprem'
    mod_data_src='nwm21-streamflow-usgs-gages-onprem'
else:
    print("Not onprem; use '-cloud' data source")
    obs_data_src='nwis-streamflow-usgs-gages-cloud'
    mod_data_src='nwm21-streamflow-usgs-gages-cloud'
print("Observed : ", obs_data_src)
print("Modeled  : ", mod_data_src)

Not onprem; use '-cloud' data source
Observed :  nwis-streamflow-usgs-gages-cloud
Modeled  :  nwm21-streamflow-usgs-gages-cloud


In [4]:
variable_of_interest = 'streamflow'
try:
    obs = cat[obs_data_src].to_dask()
    mod = cat[mod_data_src].to_dask()
except KeyError:
    print("Something wrong with dataset names.")
    raise

try:
    obs_data = obs[variable_of_interest]
    mod_data = mod[variable_of_interest].astype('float32')
except KeyError:
    print(f"{variable_of_interest} was not found in these data.")

obs_data.name = 'observed'
mod_data.name = 'predicted'    

## Step 2: Restrict to a Domain

<img src='./Eval_Analysis_Domain.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  
2) The domain over which to benchmark <span style="color:red; font-size:large"><<--You are here</span>
3) A set of statistical metrics with which to benchmark. 

Each benchmark domain is defined over specific bounds (typically space and/or time). 
Benchmark domain definitions can be defined within the notebook, or sourced from
elsewhere. For this example, we use the _Cobalt_ gages, avaliable for download on ScienceBase 
([Foks et al., 2022](https://doi.org/10.5066/P972P42Z)).  

This is essentially a list of 
stream guages in which we are interested, along with some  metadata about that gage (watershed, 
reach code, etc).  We will use this as a spatial selector to restrict the original data to only 
those gages found within this benchmarking domain.

In [5]:
cobalt = pd.read_csv(
    'https://www.sciencebase.gov/catalog/file/get/6181ac65d34e9f2789e44897?f=__disk__22%2F1a%2Fd2%2F221ad2fe9d95de17731ad35d0fc6b417a4b53ee1',
    dtype={'site_no':str, 'huc_cd':str, 'reachcode':str, 'comid':str, 'gagesII_class':str, 'aggecoregion': str}, 
    index_col='site_no'
    )
# Re-format the gage_id/site_no string value.  ex:   "1000000"  ==> "USGS-1000000"
cobalt.rename(index=lambda x: f'USGS-{x}', inplace=True)
print(f"{len(cobalt.index)} gages in this benchmark")
cobalt.head(2)

5390 gages in this benchmark


,dec_lat_va,dec_long_va,comid,reachcode,reach_meas,drain_sqkm,huc02,gagesII_class,aggecoregion,complete_yrs,n_days,nldi,swim,gfv1d1,camels
site_no,,,,,,,,,,,,,,,
USGS-01011000,47.069722,-69.079444,721640,01010002000001,53.747189,3186.8,01,Non-ref,NorthEast,33,12146,1,1,1,0
USGS-01013500,47.237500,-68.582778,724696,01010003000003,7.660419,2252.7,01,Ref,NorthEast,33,12146,1,1,1,1


So now we have a domain dataset representing the stream gages (unique `site_no` values) identifying the locations making up the spatial domain of this benchmark. While we have good meta-data for these gages (lat/lon location, HUC8 code, etc), we really will only use the list of `site_no` values to select locations out of the raw data.

## Step 3: Define Metrics

<img src='./Eval_Analysis_Metrics.svg' width=360>

Essential Benchmark Components: 
1) A set of predictions and matching observations,  
2) The domain over which to benchmark 
3) A set of statistical metrics with which to benchmark. <span style="color:red; font-size:large"><<--You are here</span>

The code to calculate the various metrics has been standardized [here](./NWM_StandardSuite_v1.ipynb). You can 
use these or write your own.  To import and use these standard definitions, run this cell:

In [6]:
%run ../../Metrics_DScore_Suite_v1.ipynb

Whether you use these functions or your own, we need to put all metric computation into a special all-encompasing 
benchmarking function--a single call which can be assigned to each gage in our domain list. This sort of arrangement 
is well-suited to parallelism with `dask`. 

If this is done well, the process will benefit enormously from task parallelism -- each gage can be given its own 
CPU to run on.  After all are done, the various results will be collected and assembled into a composite dataset. 

To achieve this, we need a single 'atomic' function that can execute independently. It will take the gage identifier 
as input and return a list of metrics.

In [14]:
## Wrapper function -- this func will be called once per gage_id, each call on its own dask worker
def compute_benchmark(gage_id):
    try:
        ## obs_data and mod_data should be globals...
        obs = obs_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series()
        mod = mod_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series().resample('1D', offset='5h').mean() 
        
        # make sure the indices match
        obs.index = obs.index.to_period('D')
        mod.index = mod.index.to_period('D')

        # merge obs and predictions; drop NaNs.
        gage_df = pd.merge(obs, mod, left_index=True, right_index=True).dropna(how='any')
        
        obs_log = np.log(gage_df['observed'].clip(lower=0.01)) # clip to remove zeros and negative values
        sim_log = np.log(gage_df['predicted'].clip(lower=0.01))
        
        scores = pd.concat([
                pd.Series([ mse(obs_log, sim_log) ], index=["mse"], dtype='float64'),
                bias_distribution_sequence(obs_log, sim_log), 
                seasonal_mse(obs_log, sim_log),
                quantile_mse(obs_log, sim_log)
                ]
            )
        scores.name=gage_id
        return scores
    except Exception as e:#<-- this is an extremely broad way to catch exceptions.  We only do it this way to ensure 
            #    that a failure on one benchmark (for a single stream gage) will not halt the entire run. 
        logging.info("Benchmark failed for %s", gage_id)
        #raise
        return None

Let's test to be sure this `compute_benchmark()` function will return data for a single gage

In [15]:
gage_id='USGS-01030350'
obs = obs_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series()
mod = mod_data.sel(gage_id=gage_id).load(scheduler='single-threaded').to_series().resample('1D', offset='5h').mean() 

# make sure the indices match
obs.index = obs.index.to_period('D')
mod.index = mod.index.to_period('D')

# merge obs and predictions; drop NaNs.
gage_df = pd.merge(obs, mod, left_index=True, right_index=True).dropna(how='any')
gage_df

/home/conda/users/4a0797bbd1f8291f89a5724f0c4063e777248062b823ac5906f8dede9cb4192f-20230118-174644-766648-50-pangeo/lib/python3.9/site-packages/dask/base.py:1367: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


,observed,predicted
time,,
2008-10-01,2.018991,7.970833
2008-10-02,2.973269,12.272500
2008-10-03,5.521785,9.787916
2008-10-04,5.323567,8.144583
2008-10-05,4.728913,5.756250
...,...,...
2020-12-27,4.643963,8.657499
2020-12-28,3.596240,9.261250
2020-12-29,2.831685,6.902916


In [9]:
compute_benchmark('USGS-01030350')

mse          1.356541
e_bias       0.181100
e_dist       0.914028
e_seq        0.261676
winter       0.087221
spring       0.101688
summer       0.549519
fall         0.618114
low          1.110189
below_avg    0.076378
above_avg    0.055341
high         0.114634
Name: USGS-01030350, dtype: float64

## Execute the Analysis 
We now need to set up a way to farm out this function, once per gage ID, to workers in a parallel cluster. `dask` will do this
using a dask '_bag_'.  

:::{sidebar}

Read more about task parallelism with Dask and how we are using dask bags [here](/dev/null)

:::

In [10]:
import os
### ### ### CONFIG CUSTOM CLUSTER HERE ### ### ###
aws_profile = 'esip-qhub'
aws_region = 'us-west-2'
aws_endpoint = f's3.{aws_region}.amazonaws.com'
  
import configparser
awsconfig = configparser.ConfigParser()
awsconfig.read(
    os.path.expanduser(r'~/.aws/credentials') # default location... if yours is elsewhere, change this.
)
# Set environment vars based on parsed awsconfig
os.environ['AWS_ACCESS_KEY_ID']     = awsconfig[aws_profile]['aws_access_key_id']    
os.environ['AWS_SECRET_ACCESS_KEY'] = awsconfig[aws_profile]['aws_secret_access_key']    
os.environ['AWS_S3_ENDPOINT']       = aws_endpoint
os.environ['AWS_DEFAULT_REGION']    = aws_region

try: 
    del os.environ['AWS_PROFILE']
except KeyError:
    pass

In [11]:
from dask_gateway import Gateway
gateway = Gateway()
options = gateway.cluster_options()
options.conda_environment='users/users-pangeo'
options.profile = 'Medium Worker'
options.environment_vars = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)
options.environment_vars.update(dict(os.environ))
cluster = gateway.new_cluster(options)
cluster.adapt(minimum=10, maximum=30)

# get the client for the cluster
client = cluster.get_client()
client.dashboard_link

'https://nebari.esipfed.org/gateway/clusters/dev.308b1bbbeddc49cb9c930757e79a7ffc/status'

In [16]:
# Set up a dask bag with the contents beging a list of the cobalt gages
import dask.bag as db
bag = db.from_sequence( 
    cobalt.index.tolist() 
).map(compute_benchmark)
results = bag.compute() 

With that big task done, we don't need `dask` parallelism any more. Let's shut down the cluster:

In [17]:
client.close(); del client
cluster.close(); del cluster

/home/conda/users/4a0797bbd1f8291f89a5724f0c4063e777248062b823ac5906f8dede9cb4192f-20230118-174644-766648-50-pangeo/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()


## Assemble the results
The `bag` now contains a collection of return values (one per call to `compute_benchmark()`).  We can massage that into a table/dataframe for easier processing: 

In [18]:
results = [i for i in results if i is not None] # Drop entries where compute_benchmark failed
results_df = pd.concat(results, axis=1).T
results_df.index.name = 'site_no'
results_df

,mse,e_bias,e_dist,e_seq,winter,spring,summer,fall,low,below_avg,above_avg,high
site_no,,,,,,,,,,,,
USGS-01011000,0.349280,0.062854,0.008747,0.277697,0.077164,0.108429,0.108192,0.055495,0.052829,0.092242,0.104895,0.099314
USGS-01013500,0.231121,0.002947,0.070471,0.157718,0.024475,0.088122,0.049798,0.068726,0.072867,0.038483,0.036119,0.083652
USGS-01015800,0.283531,0.000781,0.014034,0.268733,0.039518,0.110405,0.073111,0.060497,0.059348,0.060264,0.074423,0.089495
USGS-01017000,0.316433,0.001108,0.012394,0.302951,0.046091,0.109606,0.083467,0.077269,0.062643,0.064209,0.086774,0.102808
USGS-01017550,0.667978,0.020339,0.046881,0.600846,0.125872,0.201270,0.146279,0.194557,0.128817,0.154305,0.165163,0.219694
...,...,...,...,...,...,...,...,...,...,...,...,...
USGS-14369500,0.458882,0.008361,0.152934,0.297617,0.061887,0.084826,0.153114,0.159056,0.194792,0.084246,0.119609,0.060235
USGS-14372300,0.306279,0.010342,0.148491,0.147466,0.047533,0.045161,0.043905,0.169680,0.105135,0.101732,0.062807,0.036606
USGS-14375100,2.365585,0.529877,0.202781,1.633049,1.084581,0.929287,0.222655,0.129067,1.310581,0.822220,0.133049,0.099743


This dataframe/table can be saved to disk as a CSV. It will be used for visualizations in [other notebooks](NWM_Benchmark_Visualization.ipynb).

In [19]:
results_df.to_csv('DScore_streamflow_example.csv') ##<--- change this to a personalized filename